In [54]:
import sys
sys.path.append('..')

import os
import glob
from tqdm import tqdm_notebook as tqdm

from data.bioasq_dataset import BioASQDataset
from utilities.file_utils import Utils as utils


%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## Loading and Cleaning Data

In [2]:
data_path = os.path.join(utils.data_path.rsplit('/', 2)[0], 'data')
bio_data_path = os.path.join(data_path, 'BioASQ-67b-150ct2019')

## Dataset Analysis

In [94]:
dataset = BioASQDataset('test', True)

Loading and preprocessing file: /Users/ordonez2/Desktop/cs221/final project/domain_adaptation_QA/data/BioASQ/BioASQ-test-factoid-4b-2.json



Loading and preprocessing file: /Users/ordonez2/Desktop/cs221/final project/domain_adaptation_QA/data/BioASQ/BioASQ-test-factoid-6b-5.json



Loading and preprocessing file: /Users/ordonez2/Desktop/cs221/final project/domain_adaptation_QA/data/BioASQ/BioASQ-test-factoid-5b-3.json



Loading and preprocessing file: /Users/ordonez2/Desktop/cs221/final project/domain_adaptation_QA/data/BioASQ/BioASQ-test-factoid-5b-2.json



Loading and preprocessing file: /Users/ordonez2/Desktop/cs221/final project/domain_adaptation_QA/data/BioASQ/BioASQ-test-factoid-6b-4.json


...


Loading and preprocessing file: /Users/ordonez2/Desktop/cs221/final project/domain_adaptation_QA/data/BioASQ/BioASQ-test-factoid-6b-2.json



Loading and preprocessing file: /Users/ordonez2/Desktop/cs221/final project/domain_adaptation_QA/data/BioASQ/BioASQ-test-factoid-4b-5.json



Loading and preprocessing file: /Users/ordonez2/Desktop/cs221/final project/domain_adaptation_QA/data/BioASQ/BioASQ-test-factoid-6b-1.json



Loading and preprocessing file: /Users/ordonez2/Desktop/cs221/final project/domain_adaptation_QA/data/BioASQ/BioASQ-test-factoid-5b-1.json



Loading and preprocessing file: /Users/ordonez2/Desktop/cs221/final project/domain_adaptation_QA/data/BioASQ/BioASQ-test-factoid-4b-1.json


In [96]:
answer_found = []
for k, v in dataset.data.items():
    if v['answers'] in v['context']:
        answer_found.append(k)

In [98]:
len(answer_found)

129

In [99]:
len(dataset.data)

1671

In [86]:
question, answers, context = dataset[49]

In [100]:
question

'Which gene harbors the mutation T790M?'

In [101]:
answers

('Which gene harbors the mutation T790M?',
 ['The T790M mutation refers to the mutation in exon 20 of the EGFR gene'],
 'factoid')

In [102]:
context

'Detection of epidermal growth factor receptor mutation in lung cancer by droplet digital polymerase chain reaction. BACKGROUND: Two types of epidermal growth factor receptor (EGFR) mutations in exon 19 and exon 21 (ex19del and L858R) are prevalent in lung cancer patients and sensitive to targeted EGFR inhibition. A resistance mutation in exon 20 (T790M) has been found to accompany drug treatment when patients relapse. These three mutations are valuable companion diagnostic biomarkers for guiding personalized treatment. Quantitative polymerase chain reaction (qPCR)-based methods have been widely used in the clinic by physicians to guide treatment decisions. The aim of this study was to evaluate the technical and clinical sensitivity and specificity of the droplet digital polymerase chain reaction (ddPCR) method in detecting the three EGFR mutations in patients with lung cancer. METHODS: Genomic DNA from H1975 and PC-9 cells, as well as 92 normal human blood specimens, was used to deter

In [67]:
noanswer = [1 for k, v in dataset.data.items() if v['answers'] == None]
sum(noanswer)

0

In [68]:
noanswer

[]